# Test MMD sur des processus de Hawkes avec différents paramètres

Ce notebook génère des processus de Hawkes avec différentes configurations de paramètres
et les compare via le test MMD two-sample par batch.

**Objectif** : vérifier que le test MMD détecte correctement :
- **H0 acceptée** (p-value élevée) : deux échantillons issus de la même distribution
- **H0 rejetée** (p-value faible) : deux échantillons issus de distributions différentes

## Plan
1. Imports & helpers
2. Génération de Hawkes avec différents paramètres
3. Conversion en `Batch` pour le MMD
4. Test MMD same vs same (sanity check)
5. Test MMD entre distributions différentes
6. Étude paramétrique : variation progressive des paramètres
7. Visualisation des résultats

## 1. Imports & Helpers

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from typing import List, Dict
from itertools import combinations

from new_ltpp.data.generation.hawkes import HawkesSimulator
from new_ltpp.data.generation.simulation_manager import SimulationManager
from new_ltpp.shared_types import Batch

from new_ltpp.evaluation.statistical_testing import MMDTwoSampleTest
from new_ltpp.evaluation.statistical_testing.kernels import (
    MKernel,
    MKernelTransform,
    create_time_kernel,
    EmbeddingKernel,
    SIGKernel,
)

print("Modules importés.")

Modules importés.


In [2]:
# ---------------------------------------------------------------------------
# Helpers : simuler des Hawkes et convertir en Batch
# ---------------------------------------------------------------------------

def simulate_hawkes(
    mu, alpha, beta, dim_process, num_simulations=200,
    start_time=0, end_time=100, seed=None,
) -> List[Dict]:
    """Simule des processus de Hawkes et retourne les séquences formatées."""
    sim = HawkesSimulator(
        mu=mu, alpha=alpha, beta=beta,
        dim_process=dim_process,
        start_time=start_time, end_time=end_time, seed=seed,
    )
    manager = SimulationManager(
        simulation_func=sim.simulate,
        dim_process=dim_process,
        start_time=start_time, end_time=end_time,
    )
    return manager.bulk_simulate(num_simulations)


def sequences_to_batch(sequences: List[Dict], max_len: int | None = None) -> Batch:
    """Convertit une liste de séquences formatées en un objet Batch (padded)."""
    if max_len is None:
        max_len = max(s["seq_len"] for s in sequences)

    time_seqs, delta_seqs, type_seqs, masks = [], [], [], []

    for s in sequences:
        L = s["seq_len"]
        t = s["time_since_start"] + [0.0] * (max_len - L)
        dt = s["time_since_last_event"] + [0.0] * (max_len - L)
        tp = s["type_event"] + [0] * (max_len - L)
        m = [True] * L + [False] * (max_len - L)

        time_seqs.append(t)
        delta_seqs.append(dt)
        type_seqs.append(tp)
        masks.append(m)

    return Batch(
        time_seqs=torch.tensor(time_seqs, dtype=torch.float32),
        time_delta_seqs=torch.tensor(delta_seqs, dtype=torch.float32),
        type_seqs=torch.tensor(type_seqs, dtype=torch.long),
        valid_event_mask=torch.tensor(masks, dtype=torch.bool),
    )


def simulate_and_batch(params: dict, num_simulations=200, max_len=None) -> Batch:
    """Pipeline complet : simuler puis convertir en Batch."""
    seqs = simulate_hawkes(**params, num_simulations=num_simulations)
    return sequences_to_batch(seqs, max_len=max_len)


print("Helpers prêts.")

Helpers prêts.


## 2. Définition des configurations Hawkes

In [3]:
# ---------------------------------------------------------------------------
# Configurations de processus de Hawkes avec paramètres variés
# ---------------------------------------------------------------------------

DIM = 2  # Processus bivarié
NUM_SIM = 200  # Séquences par config

configs = {
    # --- Baseline : excitation modérée ---
    "baseline": dict(
        mu=[0.2, 0.2],
        alpha=[[0.3, 0.1], [0.1, 0.3]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
    # --- Même params (copie pour test same-vs-same) ---
    "baseline_copy": dict(
        mu=[0.2, 0.2],
        alpha=[[0.3, 0.1], [0.1, 0.3]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
    # --- Forte excitation (alpha élevé) ---
    "high_excitation": dict(
        mu=[0.2, 0.2],
        alpha=[[0.8, 0.5], [0.5, 0.8]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
    # --- Faible excitation ---
    "low_excitation": dict(
        mu=[0.2, 0.2],
        alpha=[[0.05, 0.02], [0.02, 0.05]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
    # --- Intensité de base élevée ---
    "high_baseline": dict(
        mu=[0.8, 0.8],
        alpha=[[0.3, 0.1], [0.1, 0.3]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
    # --- Décroissance lente (beta faible → mémoire longue) ---
    "slow_decay": dict(
        mu=[0.2, 0.2],
        alpha=[[0.3, 0.1], [0.1, 0.3]],
        beta=[[0.5, 0.3], [0.3, 0.5]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
    # --- Asymétrique ---
    "asymmetric": dict(
        mu=[0.1, 0.5],
        alpha=[[0.1, 0.6], [0.05, 0.2]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    ),
}

print(f"{len(configs)} configurations définies : {list(configs.keys())}")

7 configurations définies : ['baseline', 'baseline_copy', 'high_excitation', 'low_excitation', 'high_baseline', 'slow_decay', 'asymmetric']


## 3. Génération des données

In [4]:
# ---------------------------------------------------------------------------
# Simuler toutes les configurations et stocker les Batchs
# ---------------------------------------------------------------------------

raw_sequences = {}
for name, params in configs.items():
    print(f"Simulation de '{name}'...")
    raw_sequences[name] = simulate_hawkes(**params, num_simulations=NUM_SIM)
    avg_len = np.mean([s["seq_len"] for s in raw_sequences[name]])
    print(f"  → {len(raw_sequences[name])} séquences, longueur moyenne = {avg_len:.1f}")

# Padding commun pour pouvoir comparer toutes les configs entre elles
global_max_len = max(
    s["seq_len"] for seqs in raw_sequences.values() for s in seqs
)
print(f"\nMax seq length (padding commun) : {global_max_len}")

batches = {}
for name, seqs in raw_sequences.items():
    batches[name] = sequences_to_batch(seqs, max_len=global_max_len)
    print(f"Batch '{name}' : {batches[name].time_seqs.shape}")

Simulation de 'baseline'...


Simulating 200 processes: 100%|██████████| 200/200 [00:01<00:00, 142.61it/s]


  → 200 séquences, longueur moyenne = 57.9
Simulation de 'baseline_copy'...


Simulating 200 processes: 100%|██████████| 200/200 [00:01<00:00, 197.36it/s]


  → 200 séquences, longueur moyenne = 57.5
Simulation de 'high_excitation'...


...

  → 200 séquences, longueur moyenne = 215.7
Simulation de 'slow_decay'...


Simulating 200 processes: 100%|██████████| 200/200 [00:13<00:00, 15.12it/s]


  → 200 séquences, longueur moyenne = 603.0
Simulation de 'asymmetric'...


Simulating 200 processes: 100%|██████████| 200/200 [00:04<00:00, 44.96it/s]


  → 200 séquences, longueur moyenne = 104.6

Max seq length (padding commun) : 1997
Batch 'baseline' : torch.Size([200, 1997])
Batch 'baseline_copy' : torch.Size([200, 1997])
Batch 'high_excitation' : torch.Size([200, 1997])
Batch 'low_excitation' : torch.Size([200, 1997])
Batch 'high_baseline' : torch.Size([200, 1997])
Batch 'slow_decay' : torch.Size([200, 1997])
Batch 'asymmetric' : torch.Size([200, 1997])


## 4. Configuration du kernel MMD & du test

In [5]:
# ---------------------------------------------------------------------------
# Configuration des kernels MMD
# ---------------------------------------------------------------------------

# 1. M-Kernel (temps IMQ + types Embedding) avec transformation Cauchy
time_kernel_imq = create_time_kernel("imq", sigma=1.0)
type_kernel = EmbeddingKernel(num_classes=DIM, embedding_dim=16, sigma=1.0)

mkernel = MKernel(
    time_kernel=time_kernel_imq,
    type_kernel=type_kernel,
    sigma=1.0,
    transform=MKernelTransform.CAUCHY,
)

# 2. Signature Kernel
sigkernel = SIGKernel(
    static_kernel_type="rbf",
    embedding_type="linear_interpolant",
    dyadic_order=3,
    num_event_types=DIM,
)

N_PERMUTATIONS = 200

# Tests MMD avec les deux kernels
mmd_test_mk = MMDTwoSampleTest(kernel=mkernel, n_permutations=N_PERMUTATIONS)
mmd_test_sig = MMDTwoSampleTest(kernel=sigkernel, n_permutations=N_PERMUTATIONS)

print("Kernels configurés:")
print(f"  1. MKernel : IMQ time + Embedding type, transform={mkernel.transform.value}")
print(f"  2. SIGKernel : static_kernel={sigkernel.static_kernel_type}, dyadic_order={sigkernel.dyadic_order}")
print(f"  Permutations : {N_PERMUTATIONS}")

Kernels configurés:
  1. MKernel : IMQ time + Embedding type, transform=cauchy
  2. SIGKernel : static_kernel=rbf, dyadic_order=3
  Permutations : 200


## 5. Sanity Check : same vs same

On compare deux échantillons tirés avec les mêmes paramètres (baseline vs baseline_copy).
Le test devrait **ne pas rejeter** H0 (p-value > 0.05).

**On teste avec les deux kernels : MKernel (IMQ+Cauchy) et SigKernel.**

In [6]:
# ---------------------------------------------------------------------------
# Test same vs same : on ne devrait PAS rejeter H0
# ---------------------------------------------------------------------------

BATCH_SIZE = 64

def split_batch(batch: Batch, batch_size: int) -> List[Batch]:
    """Découpe un gros Batch en mini-batches."""
    n = batch.time_seqs.shape[0]
    sub_batches = []
    for i in range(0, n, batch_size):
        end = min(i + batch_size, n)
        sub_batches.append(Batch(
            time_seqs=batch.time_seqs[i:end],
            time_delta_seqs=batch.time_delta_seqs[i:end],
            type_seqs=batch.type_seqs[i:end],
            valid_event_mask=batch.valid_event_mask[i:end],
        ))
    return sub_batches


def mmd_test_by_batch(
    batch_x: Batch, batch_y: Batch, test: MMDTwoSampleTest,
    batch_size: int = 64, verbose: bool = True
) -> Dict:
    """Exécute le test MMD par batch et agrège les résultats."""
    sub_x = split_batch(batch_x, batch_size)
    sub_y = split_batch(batch_y, batch_size)
    n_pairs = min(len(sub_x), len(sub_y))

    mmd_values = []
    p_values = []

    for i in range(n_pairs):
        mmd_val = test.statistic_from_batches(sub_x[i], sub_y[i])
        p_val = test.p_value_from_batches(sub_x[i], sub_y[i])
        mmd_values.append(mmd_val)
        p_values.append(p_val)
        if verbose:
            print(f"  Batch {i+1}/{n_pairs} : MMD² = {mmd_val:.6f}, p-value = {p_val:.4f}")

    return {
        "mmd_values": mmd_values,
        "p_values": p_values,
        "mean_mmd": np.mean(mmd_values),
        "mean_p_value": np.mean(p_values),
    }


print("=" * 80)
print("SANITY CHECK : baseline vs baseline_copy (même distribution)")
print("=" * 80)

print("\n📊 Test avec MKernel (IMQ + Cauchy):")
print("-" * 80)
result_same_mk = mmd_test_by_batch(batches["baseline"], batches["baseline_copy"], mmd_test_mk, BATCH_SIZE)
print(f"\n→ MMD² moyen = {result_same_mk['mean_mmd']:.6f}")
print(f"→ p-value moyenne = {result_same_mk['mean_p_value']:.4f}")
print(f"→ Conclusion : {'H0 NON rejetée ✓' if result_same_mk['mean_p_value'] > 0.05 else 'H0 rejetée ✗ (inattendu)'}")

print("\n📊 Test avec SigKernel:")
print("-" * 80)
result_same_sig = mmd_test_by_batch(batches["baseline"], batches["baseline_copy"], mmd_test_sig, BATCH_SIZE)
print(f"\n→ MMD² moyen = {result_same_sig['mean_mmd']:.6f}")
print(f"→ p-value moyenne = {result_same_sig['mean_p_value']:.4f}")
print(f"→ Conclusion : {'H0 NON rejetée ✓' if result_same_sig['mean_p_value'] > 0.05 else 'H0 rejetée ✗ (inattendu)'}")

SANITY CHECK : baseline vs baseline_copy (même distribution)

📊 Test avec MKernel (IMQ + Cauchy):
--------------------------------------------------------------------------------


KeyboardInterrupt: 

## 6. Test MMD entre toutes les paires de distributions

On compare chaque paire de configurations distinctes avec les deux kernels.
Les paires avec des paramètres très différents devraient rejeter H0.

In [ ]:
# ---------------------------------------------------------------------------
# Test MMD pour toutes les paires de configs avec les deux kernels
# ---------------------------------------------------------------------------

config_names = [n for n in configs.keys() if n != "baseline_copy"]
pairs = list(combinations(config_names, 2))

results_mk = {}
results_sig = {}

for name_a, name_b in pairs:
    print(f"\n{'─' * 60}")
    print(f"Test : {name_a} vs {name_b}")
    print(f"{'─' * 60}")
    
    # Test avec MKernel
    print("  🔹 MKernel (IMQ + Cauchy)...")
    res_mk = mmd_test_by_batch(batches[name_a], batches[name_b], mmd_test_mk, BATCH_SIZE, verbose=False)
    results_mk[(name_a, name_b)] = res_mk
    rejected_mk = res_mk['mean_p_value'] < 0.05
    print(f"    → MMD² = {res_mk['mean_mmd']:.6f}, p-value = {res_mk['mean_p_value']:.4f}")
    print(f"    → {'H0 REJETÉE' if rejected_mk else 'H0 non rejetée'}")
    
    # Test avec SigKernel
    print("  🔹 SigKernel...")
    res_sig = mmd_test_by_batch(batches[name_a], batches[name_b], mmd_test_sig, BATCH_SIZE, verbose=False)
    results_sig[(name_a, name_b)] = res_sig
    rejected_sig = res_sig['mean_p_value'] < 0.05
    print(f"    → MMD² = {res_sig['mean_mmd']:.6f}, p-value = {res_sig['mean_p_value']:.4f}")
    print(f"    → {'H0 REJETÉE' if rejected_sig else 'H0 non rejetée'}")

## 7. Tableau récapitulatif

In [ ]:
# ---------------------------------------------------------------------------
# Résumé sous forme de tableaux
# ---------------------------------------------------------------------------

print("\n" + "=" * 90)
print("TABLEAU RÉCAPITULATIF : MKernel (IMQ + Cauchy)")
print("=" * 90)
print(f"{'Paire':<40} {'MMD²':>10} {'p-value':>10} {'Rejet H0':>10}")
print("─" * 90)

# Same vs same
label = "baseline vs baseline_copy"
print(f"{label:<40} {result_same_mk['mean_mmd']:>10.6f} {result_same_mk['mean_p_value']:>10.4f} {'Non':>10}")

# Toutes les paires
for (a, b), res in sorted(results_mk.items(), key=lambda x: x[1]['mean_p_value']):
    label = f"{a} vs {b}"
    rejected = "Oui" if res['mean_p_value'] < 0.05 else "Non"
    print(f"{label:<40} {res['mean_mmd']:>10.6f} {res['mean_p_value']:>10.4f} {rejected:>10}")

print("\n" + "=" * 90)
print("TABLEAU RÉCAPITULATIF : SigKernel")
print("=" * 90)
print(f"{'Paire':<40} {'MMD²':>10} {'p-value':>10} {'Rejet H0':>10}")
print("─" * 90)

# Same vs same
print(f"{label:<40} {result_same_sig['mean_mmd']:>10.6f} {result_same_sig['mean_p_value']:>10.4f} {'Non':>10}")

# Toutes les paires
for (a, b), res in sorted(results_sig.items(), key=lambda x: x[1]['mean_p_value']):
    label = f"{a} vs {b}"
    rejected = "Oui" if res['mean_p_value'] < 0.05 else "Non"
    print(f"{label:<40} {res['mean_mmd']:>10.6f} {res['mean_p_value']:>10.4f} {rejected:>10}")

## 8. Étude paramétrique : variation progressive de alpha

On fixe tout sauf alpha (excitation) qu'on fait varier de 0.05 à 0.9.
On compare chaque config au baseline pour voir à quel point le MMD diverge.

In [ ]:
# ---------------------------------------------------------------------------
# Variation progressive d'alpha avec les deux kernels
# ---------------------------------------------------------------------------

alpha_values = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alpha_results_mk = []
alpha_results_sig = []

baseline_batch = batches["baseline"]

for alpha_diag in alpha_values:
    off_diag = alpha_diag * 0.3  # cross-excitation proportionnelle
    params = dict(
        mu=[0.2, 0.2],
        alpha=[[alpha_diag, off_diag], [off_diag, alpha_diag]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    )
    seqs = simulate_hawkes(**params, num_simulations=NUM_SIM)
    batch_alpha = sequences_to_batch(seqs, max_len=global_max_len)

    # Test MMD avec MKernel
    mmd_val_mk = mmd_test_mk.statistic_from_batches(baseline_batch, batch_alpha)
    p_val_mk = mmd_test_mk.p_value_from_batches(baseline_batch, batch_alpha)
    alpha_results_mk.append({"alpha": alpha_diag, "mmd": mmd_val_mk, "p_value": p_val_mk})
    
    # Test MMD avec SigKernel
    mmd_val_sig = mmd_test_sig.statistic_from_batches(baseline_batch, batch_alpha)
    p_val_sig = mmd_test_sig.p_value_from_batches(baseline_batch, batch_alpha)
    alpha_results_sig.append({"alpha": alpha_diag, "mmd": mmd_val_sig, "p_value": p_val_sig})
    
    print(f"alpha={alpha_diag:.2f}")
    print(f"  MKernel  : MMD²={mmd_val_mk:.6f}, p-value={p_val_mk:.4f}")
    print(f"  SigKernel: MMD²={mmd_val_sig:.6f}, p-value={p_val_sig:.4f}")

print("\nDone.")

## 9. Visualisations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

alphas = [r["alpha"] for r in alpha_results_mk]

# MKernel results
mmds_mk = [r["mmd"] for r in alpha_results_mk]
pvals_mk = [r["p_value"] for r in alpha_results_mk]

# SigKernel results
mmds_sig = [r["mmd"] for r in alpha_results_sig]
pvals_sig = [r["p_value"] for r in alpha_results_sig]

# --- MKernel : MMD² vs alpha ---
axes[0, 0].plot(alphas, mmds_mk, "o-", color="steelblue", linewidth=2, markersize=8, label="MKernel")
axes[0, 0].axvline(x=0.3, color="red", linestyle="--", alpha=0.7, label="baseline α=0.3")
axes[0, 0].set_xlabel("α (diagonale excitation)", fontsize=12)
axes[0, 0].set_ylabel("MMD²", fontsize=12)
axes[0, 0].set_title("MKernel (IMQ+Cauchy) : MMD² vs baseline", fontsize=13)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# --- MKernel : p-value vs alpha ---
axes[0, 1].plot(alphas, pvals_mk, "o-", color="darkorange", linewidth=2, markersize=8, label="MKernel")
axes[0, 1].axhline(y=0.05, color="red", linestyle="--", alpha=0.7, label="seuil α=0.05")
axes[0, 1].axvline(x=0.3, color="gray", linestyle=":", alpha=0.7, label="baseline α=0.3")
axes[0, 1].set_xlabel("α (diagonale excitation)", fontsize=12)
axes[0, 1].set_ylabel("p-value", fontsize=12)
axes[0, 1].set_title("MKernel : p-value du test MMD", fontsize=13)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# --- SigKernel : MMD² vs alpha ---
axes[1, 0].plot(alphas, mmds_sig, "s-", color="forestgreen", linewidth=2, markersize=8, label="SigKernel")
axes[1, 0].axvline(x=0.3, color="red", linestyle="--", alpha=0.7, label="baseline α=0.3")
axes[1, 0].set_xlabel("α (diagonale excitation)", fontsize=12)
axes[1, 0].set_ylabel("MMD²", fontsize=12)
axes[1, 0].set_title("SigKernel : MMD² vs baseline", fontsize=13)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# --- SigKernel : p-value vs alpha ---
axes[1, 1].plot(alphas, pvals_sig, "s-", color="crimson", linewidth=2, markersize=8, label="SigKernel")
axes[1, 1].axhline(y=0.05, color="red", linestyle="--", alpha=0.7, label="seuil α=0.05")
axes[1, 1].axvline(x=0.3, color="gray", linestyle=":", alpha=0.7, label="baseline α=0.3")
axes[1, 1].set_xlabel("α (diagonale excitation)", fontsize=12)
axes[1, 1].set_ylabel("p-value", fontsize=12)
axes[1, 1].set_title("SigKernel : p-value du test MMD", fontsize=13)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------------------------
# Heatmaps des p-values entre toutes les paires (pour les deux kernels)
# ---------------------------------------------------------------------------

names = config_names
n = len(names)

# Matrices pour MKernel
pval_matrix_mk = np.ones((n, n))
mmd_matrix_mk = np.zeros((n, n))

# Matrices pour SigKernel
pval_matrix_sig = np.ones((n, n))
mmd_matrix_sig = np.zeros((n, n))

# Remplissage MKernel
for (a, b), res in results_mk.items():
    i, j = names.index(a), names.index(b)
    pval_matrix_mk[i, j] = res["mean_p_value"]
    pval_matrix_mk[j, i] = res["mean_p_value"]
    mmd_matrix_mk[i, j] = res["mean_mmd"]
    mmd_matrix_mk[j, i] = res["mean_mmd"]

# Remplissage SigKernel
for (a, b), res in results_sig.items():
    i, j = names.index(a), names.index(b)
    pval_matrix_sig[i, j] = res["mean_p_value"]
    pval_matrix_sig[j, i] = res["mean_p_value"]
    mmd_matrix_sig[i, j] = res["mean_mmd"]
    mmd_matrix_sig[j, i] = res["mean_mmd"]

fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# --- MKernel : p-values ---
im1 = axes[0, 0].imshow(pval_matrix_mk, cmap="RdYlGn", vmin=0, vmax=1)
axes[0, 0].set_xticks(range(n))
axes[0, 0].set_yticks(range(n))
axes[0, 0].set_xticklabels(names, rotation=45, ha="right", fontsize=9)
axes[0, 0].set_yticklabels(names, fontsize=9)
axes[0, 0].set_title("MKernel (IMQ+Cauchy) : p-values", fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(n):
        axes[0, 0].text(j, i, f"{pval_matrix_mk[i,j]:.3f}", ha="center", va="center", fontsize=8)
plt.colorbar(im1, ax=axes[0, 0], shrink=0.8)

# --- MKernel : MMD² ---
im2 = axes[0, 1].imshow(mmd_matrix_mk, cmap="YlOrRd")
axes[0, 1].set_xticks(range(n))
axes[0, 1].set_yticks(range(n))
axes[0, 1].set_xticklabels(names, rotation=45, ha="right", fontsize=9)
axes[0, 1].set_yticklabels(names, fontsize=9)
axes[0, 1].set_title("MKernel (IMQ+Cauchy) : MMD²", fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(n):
        axes[0, 1].text(j, i, f"{mmd_matrix_mk[i,j]:.4f}", ha="center", va="center", fontsize=8)
plt.colorbar(im2, ax=axes[0, 1], shrink=0.8)

# --- SigKernel : p-values ---
im3 = axes[1, 0].imshow(pval_matrix_sig, cmap="RdYlGn", vmin=0, vmax=1)
axes[1, 0].set_xticks(range(n))
axes[1, 0].set_yticks(range(n))
axes[1, 0].set_xticklabels(names, rotation=45, ha="right", fontsize=9)
axes[1, 0].set_yticklabels(names, fontsize=9)
axes[1, 0].set_title("SigKernel : p-values", fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(n):
        axes[1, 0].text(j, i, f"{pval_matrix_sig[i,j]:.3f}", ha="center", va="center", fontsize=8)
plt.colorbar(im3, ax=axes[1, 0], shrink=0.8)

# --- SigKernel : MMD² ---
im4 = axes[1, 1].imshow(mmd_matrix_sig, cmap="YlOrRd")
axes[1, 1].set_xticks(range(n))
axes[1, 1].set_yticks(range(n))
axes[1, 1].set_xticklabels(names, rotation=45, ha="right", fontsize=9)
axes[1, 1].set_yticklabels(names, fontsize=9)
axes[1, 1].set_title("SigKernel : MMD²", fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(n):
        axes[1, 1].text(j, i, f"{mmd_matrix_sig[i,j]:.4f}", ha="center", va="center", fontsize=8)
plt.colorbar(im4, ax=axes[1, 1], shrink=0.8)

plt.tight_layout()
plt.show()

## 10. Étude de puissance : variation de mu

On fait varier mu (intensité de base) en gardant alpha et beta fixes.

In [ ]:
# ---------------------------------------------------------------------------
# Variation progressive de mu avec les deux kernels
# ---------------------------------------------------------------------------

mu_values = [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0, 1.5]
mu_results_mk = []
mu_results_sig = []

for mu_val in mu_values:
    params = dict(
        mu=[mu_val, mu_val],
        alpha=[[0.3, 0.1], [0.1, 0.3]],
        beta=[[2.0, 1.0], [1.0, 2.0]],
        dim_process=DIM,
        start_time=0, end_time=100,
    )
    seqs = simulate_hawkes(**params, num_simulations=NUM_SIM)
    batch_mu = sequences_to_batch(seqs, max_len=global_max_len)

    # Test avec MKernel
    mmd_val_mk = mmd_test_mk.statistic_from_batches(baseline_batch, batch_mu)
    p_val_mk = mmd_test_mk.p_value_from_batches(baseline_batch, batch_mu)
    mu_results_mk.append({"mu": mu_val, "mmd": mmd_val_mk, "p_value": p_val_mk})
    
    # Test avec SigKernel
    mmd_val_sig = mmd_test_sig.statistic_from_batches(baseline_batch, batch_mu)
    p_val_sig = mmd_test_sig.p_value_from_batches(baseline_batch, batch_mu)
    mu_results_sig.append({"mu": mu_val, "mmd": mmd_val_sig, "p_value": p_val_sig})
    
    print(f"mu={mu_val:.2f}")
    print(f"  MKernel  : MMD²={mmd_val_mk:.6f}, p-value={p_val_mk:.4f}")
    print(f"  SigKernel: MMD²={mmd_val_sig:.6f}, p-value={p_val_sig:.4f}")

print("\nDone.")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

mus = [r["mu"] for r in mu_results_mk]

# MKernel results
mmds_mu_mk = [r["mmd"] for r in mu_results_mk]
pvals_mu_mk = [r["p_value"] for r in mu_results_mk]

# SigKernel results
mmds_mu_sig = [r["mmd"] for r in mu_results_sig]
pvals_mu_sig = [r["p_value"] for r in mu_results_sig]

# --- MKernel : MMD² vs mu ---
axes[0, 0].plot(mus, mmds_mu_mk, "s-", color="steelblue", linewidth=2, markersize=8, label="MKernel")
axes[0, 0].axvline(x=0.2, color="red", linestyle="--", alpha=0.7, label="baseline μ=0.2")
axes[0, 0].set_xlabel("μ (intensité de base)", fontsize=12)
axes[0, 0].set_ylabel("MMD²", fontsize=12)
axes[0, 0].set_title("MKernel (IMQ+Cauchy) : MMD² vs baseline", fontsize=13)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# --- MKernel : p-value vs mu ---
axes[0, 1].plot(mus, pvals_mu_mk, "s-", color="darkorange", linewidth=2, markersize=8, label="MKernel")
axes[0, 1].axhline(y=0.05, color="red", linestyle="--", alpha=0.7, label="seuil α=0.05")
axes[0, 1].axvline(x=0.2, color="gray", linestyle=":", alpha=0.7, label="baseline μ=0.2")
axes[0, 1].set_xlabel("μ (intensité de base)", fontsize=12)
axes[0, 1].set_ylabel("p-value", fontsize=12)
axes[0, 1].set_title("MKernel : p-value du test MMD", fontsize=13)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# --- SigKernel : MMD² vs mu ---
axes[1, 0].plot(mus, mmds_mu_sig, "^-", color="forestgreen", linewidth=2, markersize=8, label="SigKernel")
axes[1, 0].axvline(x=0.2, color="red", linestyle="--", alpha=0.7, label="baseline μ=0.2")
axes[1, 0].set_xlabel("μ (intensité de base)", fontsize=12)
axes[1, 0].set_ylabel("MMD²", fontsize=12)
axes[1, 0].set_title("SigKernel : MMD² vs baseline", fontsize=13)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# --- SigKernel : p-value vs mu ---
axes[1, 1].plot(mus, pvals_mu_sig, "^-", color="crimson", linewidth=2, markersize=8, label="SigKernel")
axes[1, 1].axhline(y=0.05, color="red", linestyle="--", alpha=0.7, label="seuil α=0.05")
axes[1, 1].axvline(x=0.2, color="gray", linestyle=":", alpha=0.7, label="baseline μ=0.2")
axes[1, 1].set_xlabel("μ (intensité de base)", fontsize=12)
axes[1, 1].set_ylabel("p-value", fontsize=12)
axes[1, 1].set_title("SigKernel : p-value du test MMD", fontsize=13)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Résumé final

In [ ]:
print("=" * 90)
print("RÉSUMÉ DES TESTS MMD")
print("=" * 90)

print("\n" + "─" * 90)
print("📊 MKERNEL (IMQ + Cauchy Transform)")
print("─" * 90)

print("\n--- Sanity check (même distribution) ---")
print(f"baseline vs baseline_copy : p-value = {result_same_mk['mean_p_value']:.4f}")
print(f"  → {'PASS : H0 non rejetée ✓' if result_same_mk['mean_p_value'] > 0.05 else 'FAIL : H0 rejetée ✗ (inattendu)'}")

print("\n--- Paires avec distributions différentes ---")
n_rejected_mk = sum(1 for res in results_mk.values() if res['mean_p_value'] < 0.05)
n_total = len(results_mk)
print(f"H0 rejetée dans {n_rejected_mk}/{n_total} paires")

print("\n--- Étude alpha (excitation) ---")
for r in alpha_results_mk:
    status = "✓ rejeté" if r['p_value'] < 0.05 else "  non rejeté"
    baseline_marker = " ← baseline" if abs(r['alpha'] - 0.3) < 0.01 else ""
    print(f"  α={r['alpha']:.2f} : p={r['p_value']:.4f} {status}{baseline_marker}")

print("\n--- Étude mu (intensité de base) ---")
for r in mu_results_mk:
    status = "✓ rejeté" if r['p_value'] < 0.05 else "  non rejeté"
    baseline_marker = " ← baseline" if abs(r['mu'] - 0.2) < 0.01 else ""
    print(f"  μ={r['mu']:.2f} : p={r['p_value']:.4f} {status}{baseline_marker}")

print("\n" + "─" * 90)
print("📊 SIGKERNEL")
print("─" * 90)

print("\n--- Sanity check (même distribution) ---")
print(f"baseline vs baseline_copy : p-value = {result_same_sig['mean_p_value']:.4f}")
print(f"  → {'PASS : H0 non rejetée ✓' if result_same_sig['mean_p_value'] > 0.05 else 'FAIL : H0 rejetée ✗ (inattendu)'}")

print("\n--- Paires avec distributions différentes ---")
n_rejected_sig = sum(1 for res in results_sig.values() if res['mean_p_value'] < 0.05)
print(f"H0 rejetée dans {n_rejected_sig}/{n_total} paires")

print("\n--- Étude alpha (excitation) ---")
for r in alpha_results_sig:
    status = "✓ rejeté" if r['p_value'] < 0.05 else "  non rejeté"
    baseline_marker = " ← baseline" if abs(r['alpha'] - 0.3) < 0.01 else ""
    print(f"  α={r['alpha']:.2f} : p={r['p_value']:.4f} {status}{baseline_marker}")

print("\n--- Étude mu (intensité de base) ---")
for r in mu_results_sig:
    status = "✓ rejeté" if r['p_value'] < 0.05 else "  non rejeté"
    baseline_marker = " ← baseline" if abs(r['mu'] - 0.2) < 0.01 else ""
    print(f"  μ={r['mu']:.2f} : p={r['p_value']:.4f} {status}{baseline_marker}")

print("\n" + "=" * 90)
print("COMPARAISON DES DEUX KERNELS")
print("=" * 90)
print(f"\nStabilité (sanity check):")
print(f"  MKernel  : p-value = {result_same_mk['mean_p_value']:.4f}")
print(f"  SigKernel: p-value = {result_same_sig['mean_p_value']:.4f}")
print(f"\nPuissance de détection (paires différentes):")
print(f"  MKernel  : {n_rejected_mk}/{n_total} rejets")
print(f"  SigKernel: {n_rejected_sig}/{n_total} rejets")
print("\n" + "=" * 90)